# Imports

In [1]:
import schemes as sc
from model_cifar import CIFAR_PAQ_COMP,CIFAR_PAQ_COMP_ADAPTIVE 



from load_dataset import Dataset
from datetime import datetime
import time
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler


d:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\FedPAQ_env\lib\site-packages\julia\juliainfo.py:93: UserWarning: julia warned:
The latest version of Julia in the `release` channel is 1.11.1+0.x64.w64.mingw32. You currently have `1.10.5+0.x64.w64.mingw32` installed. Run:

  juliaup update

in your terminal shell to install Julia 1.11.1+0.x64.w64.mingw32 and update the `release` channel to that version.
  warnings.warn("{} warned:\n{}".format(julia, stderr))


# Phase 1

## Loading Cifar dataset

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import numpy as np


In [ ]:
cifar_location = r'D:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\cifar-10\cifar-10-python'

class CIFAR10Dataset:
    def __init__(self, iid=True, batch_size=128):
        self.batch_size = batch_size
        self.iid = iid
        self.train_loader = None
        self.test_loader = None

    def load_data(self):
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        # Download and load the training data
        trainset = torchvision.datasets.CIFAR10(root=cifar_location, train=True, download=False, transform=transform)
        
        # Download and load the test data
        testset = torchvision.datasets.CIFAR10(root=cifar_location, train=False, download=False, transform=transform)

        # Calculate number of batches
        num_batches = len(trainset.data) // self.batch_size

        # Truncate the dataset to a multiple of the batch size
        train_x = trainset.data[:num_batches * self.batch_size]
        train_y = np.array(trainset.targets[:num_batches * self.batch_size])

        # Reshape into batches
        train_x_batches = train_x.reshape(num_batches, self.batch_size, *train_x.shape[1:])
        train_y_batches = train_y.reshape(num_batches, self.batch_size)
        
        return trainset.targets, trainset.data, testset.targets, testset.data

    def load_data_2(self):
        batch_size = self.batch_size
        trainset = torchvision.datasets.CIFAR10(root=cifar_location, train=True, download=False)
        testset = torchvision.datasets.CIFAR10(root=cifar_location, train=False, download=False)

        self.train_x, self.test_x, self.train_y, self.test_y = trainset.data, testset.data, trainset.targets, testset.targets

        self.train_x = np.array([self.train_x[n:n+batch_size] for n in range(0, len(self.train_x)-batch_size, batch_size)])/255.0
        self.test_x = np.array([self.test_x[n:n+batch_size] for n in range(0, len(self.test_x)-batch_size, batch_size)])/255.0
        self.train_y = np.array([self.train_y[n:n+batch_size] for n in range(0, len(self.train_y)-batch_size, batch_size)])
        self.test_y = np.array([self.test_y[n:n+batch_size] for n in range(0, len(self.test_y)-batch_size, batch_size)])

        self.train_x, self.train_y, self.test_x, self.test_y = torch.from_numpy(self.train_x), torch.from_numpy(self.train_y), torch.from_numpy(self.test_x), torch.from_numpy(self.test_y)

        return self.train_x, self.train_y, self.test_x, self.test_y

: 

### Training Code using Scheme

In [ ]:
number_of_clients = 390
aggregate_epochs = 10
local_epochs = 3
r = 0.5

current_time = datetime.now().time()
epoch_time = time.time()
tic = time.time()
filename = f"saved_models_{epoch_time}"



dataset = CIFAR10Dataset(iid=True, batch_size=128)
train_x, train_y, test_x, test_y = dataset.load_data_2()
train_x = train_x.permute(0, 1, 4, 2, 3)
test_x = test_x.permute(0, 1, 4, 2, 3)

train_x = train_x.float()
test_x = test_x.float()
train_y = torch.tensor(train_y, dtype=torch.long)
test_y = torch.tensor(test_y, dtype=torch.long)
print(train_x.shape)
cmp = sc.FP8()

c_8 = CIFAR_PAQ_COMP(cmp = cmp, filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
train_x, train_y = c_8.client_generator(train_x, train_y)
result = c_8.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y})
print("Time Taken: ", time.time()-tic)


C:\Users\Asus\AppData\Local\Temp\ipykernel_43628\3063256684.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_y = torch.tensor(train_y, dtype=torch.long)
C:\Users\Asus\AppData\Local\Temp\ipykernel_43628\3063256684.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_y = torch.tensor(test_y, dtype=torch.long)


torch.Size([390, 128, 3, 32, 32])


  0%|          | 0/195 [00:00<?, ?it/s]d:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\FedPAQ_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\FedPAQ_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
{'Epoch': 1, 'Loss': 0.59534}:  16%|█▋        | 32/195 [09:09<47:24, 17.45s/it] 

SystemError: deallocated bytearray object has exported buffers

{'Epoch': 1, 'Loss': 0.5867}:  17%|█▋        | 33/195 [09:23<43:58, 16.29s/it] 

In [ ]:
train_x.shape

torch.Size([390, 1, 128, 3, 32, 32])

In [ ]:
m = torch.nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
predicted_labels = torch.argmax(output, dim=1)
predicted_labels

tensor([0, 0])

### Result Saving

In [ ]:
file_path_1 = r'E:\MS_Thesis\Pre-defense-june\Results_for_update\proposed_variations.csv'


result =  pd.DataFrame(result, columns=m_8.cell_column_names)
display(result)
if os.path.exists(file_path_1):
    # Read the existing CSV file
    existing_data = pd.read_csv(file_path_1)
    
    # Identify the last epoch in the current run
    last_epoch = result['Epoch'].max()
    
    # Iterate over the new results and update the existing ones if necessary
    for index, row in result.iterrows():
        scheme_name = row['scheme_name']
        epoch = row['Epoch']
        new_acc = row['Acc']
        
        # Check if the scheme_name and epoch already exist in the existing data
        existing_entry = existing_data[(existing_data['scheme_name'] == scheme_name) & (existing_data['Epoch'] == epoch)]
        
        if not existing_entry.empty:
            # Check if the new accuracy at the last epoch is greater than the existing one
            if epoch == last_epoch:
                existing_last_epoch_entry = existing_data[(existing_data['scheme_name'] == scheme_name) & (existing_data['Epoch'] == last_epoch)]
                if new_acc > existing_last_epoch_entry['Acc'].values[0]:
                    # Update the existing entry
                    existing_data.loc[(existing_data['scheme_name'] == scheme_name) & (existing_data['Epoch'] == epoch), ['Loss', 'Acc']] = row[['Loss', 'Acc']]
        else:
            # Append the new row to the existing data
            # existing_data = existing_data.append(row, ignore_index=True)
            existing_data = pd.concat([existing_data, pd.DataFrame([row])], ignore_index=True)

    # Save the updated data back to the CSV file
    existing_data.to_csv(file_path_1, index=False)
else:
    # If the file does not exist, save the new result as a new CSV file
    result.to_csv(file_path_1, index=False)


# Phase 2

In [3]:
number_of_clients = 328
aggregate_epochs = 10
local_epochs = 3
r = 0.5


current_time = datetime.now().time()
tic = datetime.now()
filename = "saved_models_x"
train_x, train_y, test_x, test_y = Dataset().load_csv()

cmp_list = [sc.customFreq_2(),sc.QSGD(2,True),sc.Custom_linear_16()]
d = {}

for cmp in cmp_list:
    name, acc, time, eff, combined = sc.get_time_efficiency_acc(cmp)
    d[cmp] = eff
    
print(d)
sorted_schemes = dict(sorted(d.items(), key=lambda item: item[1], reverse=True))
print(sorted_schemes.keys())

m_8 = MNIST_PAQ_COMP_ADAPTIVE(cmp_list=list(sorted_schemes.keys()), filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
train_x, train_y = m_8.client_generator(train_x, train_y)
agg_weights = m_8.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y})
print("Time Taken: ", datetime.now()-tic)

{<schemes.customFreq_2 object at 0x000002A4B1EDAEC0>: 0.275862069, <schemes.QSGD object at 0x000002A4B1F3B370>: 0.528846154, <schemes.Custom_linear_16 object at 0x000002A4B1EBF220>: 0.413793103}
dict_keys([<schemes.QSGD object at 0x000002A4B1F3B370>, <schemes.Custom_linear_16 object at 0x000002A4B1EBF220>, <schemes.customFreq_2 object at 0x000002A4B1EDAEC0>])


  0%|          | 0/164 [00:00<?, ?it/s]e:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\model.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  weight_array = np.array(weights)
e:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\model.py:254: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30105}: 100%|██████████| 164/164 [00:27<00:00,  5.97it/s]


Test Accuracy: 0.10106
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 2, 'Loss': 1.97014}: 100%|██████████| 164/164 [00:25<00:00,  6.45it/s]


Test Accuracy: 0.43947
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


...

Test Accuracy: 0.93191
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 9, 'Loss': 1.51883}: 100%|██████████| 164/164 [00:26<00:00,  6.09it/s]


Test Accuracy: 0.94026
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 10, 'Loss': 1.51467}: 100%|██████████| 164/164 [00:27<00:00,  5.95it/s]


Test Accuracy: 0.93872
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  0:04:42.358223


In [8]:
number_of_clients = 328
aggregate_epochs = 10
local_epochs = 3
r = 0.5


current_time = datetime.now().time()
tic = datetime.now()
filename = "saved_models_x"
train_x, train_y, test_x, test_y = Dataset().load_csv()

# cmp_list = [sc.customFreq_2(),sc.QSGD(2,True),sc.Custom_linear_16()]
# cmp_list = [sc.customFreq_2(),sc.customFreq_2(),sc.customFreq_2()]
cmp_list = [sc.customFreq_1_5(),sc.customFreq_1_5(),sc.customFreq_1_5()]

d = {}

for cmp in cmp_list:
    name, acc, time, eff, combined = sc.get_time_efficiency_acc(cmp)
    d[cmp] = eff
    
print(d)
sorted_schemes = dict(sorted(d.items(), key=lambda item: item[1], reverse=True))
print(sorted_schemes.keys())

m_8 = MNIST_PAQ_COMP_ADAPTIVE(cmp_list=list(sorted_schemes.keys()), filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
train_x, train_y = m_8.client_generator(train_x, train_y)
agg_weights,client_details = m_8.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y},logging=True)
print("Time Taken: ", datetime.now()-tic)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [6]:


# Step 1: Convert dictionary to DataFrame
df = pd.DataFrame.from_dict(client_details, orient='index')

# Step 2: Rename the columns
df.columns = ['Bandwidth', 'Traffic', 'Latency', 'Compression']

# Step 3: Save to CSV file
df.to_csv(r'D:\MS_Thesis\Pre-defense-june\Results_for_update\client_latency_data_customFreq_1_5.csv', index=True)

# Optional: Print the head of the DataFrame to verify
average_latency = df['Latency'].mean()
print(f'Average Latency Acc: {average_latency:.2f}')


Average Latency Acc: 0.16


# Latency
customFreq_2 0.20
Adaptive 0.13
QSGD 0.12
FP8 0.34
Custom_linear 0.18
Custom_linear_16 0.16


# Hyperparameter study

In [6]:
number_of_clients = 328
aggregate_epochs = 10
local_epochs = 3
r = 0.5


current_time = datetime.now().time()
tic = datetime.now()
filename = "saved_models_x"
train_x, train_y, test_x, test_y = Dataset().load_csv()

cmp_list = [sc.customFreq_2(),sc.QSGD(2,True),sc.Custom_linear_16()]
# cmp_list = [sc.customFreq_2(),sc.customFreq_2(),sc.customFreq_2()]
# cmp_list = [sc.customFreq_1_5(),sc.customFreq_1_5(),sc.customFreq_1_5()]

d = {}

w_acc= 0.3
w_time = 0.3
w_eff = 0.2

for cmp in cmp_list:
    name, acc, time, eff, combined = sc.get_time_efficiency_acc(cmp, w_acc=w_acc, w_time=w_time, w_eff=w_eff)
    d[cmp] = eff
    
print(d)
sorted_schemes = dict(sorted(d.items(), key=lambda item: item[1], reverse=True))
print(sorted_schemes.keys())

m_8 = MNIST_PAQ_COMP_ADAPTIVE(cmp_list=list(sorted_schemes.keys()), filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
train_x, train_y = m_8.client_generator(train_x, train_y)
agg_weights,client_details = m_8.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y},logging=True)
print("Time Taken: ", datetime.now()-tic)


# Step 1: Convert dictionary to DataFrame
df = pd.DataFrame.from_dict(client_details, orient='index')

# Step 2: Rename the columns
df.columns = ['Bandwidth', 'Traffic', 'Latency', 'Compression']

# Step 3: Save to CSV file
df.to_csv(rf'D:\MS_Thesis\Pre-defense-june\hyperparameter_tuning\client_latency_data_w_acc{w_acc}_w_time_{w_time}_w_eff_{w_eff}.csv', index=True)

# Optional: Print the head of the DataFrame to verify
average_latency = df['Latency'].mean()
print(f'Average Latency Acc: {average_latency:.2f}')


{<schemes.customFreq_2 object at 0x0000016B2079DAE0>: 0.275862069, <schemes.QSGD object at 0x0000016B2079E0B0>: 0.528846154, <schemes.Custom_linear_16 object at 0x0000016B2079D8A0>: 0.413793103}
dict_keys([<schemes.QSGD object at 0x0000016B2079E0B0>, <schemes.Custom_linear_16 object at 0x0000016B2079D8A0>, <schemes.customFreq_2 object at 0x0000016B2079DAE0>])


  0%|          | 0/164 [00:00<?, ?it/s]d:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\model.py:249: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  weight_array = np.array(weights)
d:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\model.py:256: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30108}: 100%|██████████| 164/164 [00:20<00:00,  7.92it/s]


Test Accuracy: 0.09963
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 2, 'Loss': 1.98181}: 100%|██████████| 164/164 [00:20<00:00,  8.07it/s]


Test Accuracy: 0.40399
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


...

Test Accuracy: 0.94417
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 9, 'Loss': 1.5063}: 100%|██████████| 164/164 [00:20<00:00,  7.82it/s] 


Test Accuracy: 0.9465
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 10, 'Loss': 1.50474}: 100%|██████████| 164/164 [00:20<00:00,  7.90it/s]


Test Accuracy: 0.94789
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  0:03:42.532847
Average Latency Acc: 0.12


# hyperparameter 